In [1]:
from models.DistMult import DistMult
from utils.loaders import load_data
import pickle
import torch
import numpy as np
from sklearn.utils import shuffle
from utils.evaluation_metrics import SRR

In [2]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\Sparse_dictionary.pickle', 'rb') as handle:
    BCE_dictionary = pickle.load(handle)

entities = int(len(lookup)/2)

In [3]:
number_of_batches = 5
number_of_epochs = 20
x = shuffle(data)
test_data = x[:50] #just limit data to the first 100 rows

In [4]:
all_objects, all_relationships, all_subjects = np.split(test_data, 3, axis=1)
all_objects = torch.LongTensor(all_objects)
all_subjects = torch.squeeze(torch.LongTensor(all_subjects))
# all_relationships = torch.LongTensor([a+1 for a in all_relationships])
all_relationships = torch.LongTensor(all_relationships)

In [23]:
model = DistMult(num_entities = entities, embedding_dim=100, num_relations=4)
optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(number_of_epochs):


        #training stage 
        model.train()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            subj = torch.squeeze(torch.LongTensor(subjects[index]))

            optimiser.zero_grad()
            pred = model.forward(e1 = obj, rel = rel)
            loss = model.loss(pred, subj)
            loss.backward()
            optimiser.step()


        #evaluation stage
        model.eval()
        objects, subjects, relationships  = load_data(test_data, number_of_batches)
        total_sum_reciporical_rank = torch.zeros(1)

        for index in range(number_of_batches):

            obj = torch.LongTensor(objects[index])
            rel = torch.LongTensor(relationships[index])
            targets = torch.LongTensor(subjects[index])

            predictions = model.forward(e1 = obj, rel = rel)
            srr = SRR(predictions, targets) 
            total_sum_reciporical_rank = total_sum_reciporical_rank + srr

        print('mean reciporical rank is...')
        print(total_sum_reciporical_rank / len(test_data))



mean reciporical rank is...
tensor([0.0001])
mean reciporical rank is...
tensor([0.0003])
mean reciporical rank is...
tensor([0.0007])
mean reciporical rank is...
tensor([0.0024])
mean reciporical rank is...
tensor([0.0124])
mean reciporical rank is...
tensor([0.0304])
mean reciporical rank is...
tensor([0.0904])
mean reciporical rank is...
tensor([0.1662])
mean reciporical rank is...
tensor([0.2845])
mean reciporical rank is...
tensor([0.4244])
mean reciporical rank is...
tensor([0.5562])
mean reciporical rank is...
tensor([0.7000])
mean reciporical rank is...
tensor([0.8373])
mean reciporical rank is...
tensor([0.9104])
mean reciporical rank is...
tensor([0.9263])
mean reciporical rank is...
tensor([0.9644])
mean reciporical rank is...
tensor([0.9850])
mean reciporical rank is...
tensor([1.])
mean reciporical rank is...
tensor([1.])
mean reciporical rank is...
tensor([1.])


50

In [6]:
model.eval()

predictions = model.forward(
    e1=torch.LongTensor(objects[0]), rel=torch.LongTensor(relationships[0])
)
targets = torch.LongTensor(subjects[0])

In [8]:
Sum_reciporical_ranks = SRR(predictions, targets)

In [11]:
Sum_reciporical_ranks/len(subjects[0])

tensor(1.)

tensor([0.])